# VGG16

Construção do modelo vgg16. As feed forward layers foram retreinadas para o probelma das assinaturas enquanto os pesos das camadas anteriores foram reaproveitados de um modelo já treinado para identificação de objetos (os pesos estão no arquivo weights.npz).
Nenhuma entrada é necessária, basta executar.

Obs: os trechos comentados são para algumas alterações futuras, não possuem nenhuma importância no momento e podem ser ignorados

In [ ]:
# =============================================================================
from __future__ import print_function
import numpy as np
import tensorflow as tf
# =============================================================================
num_classes = 2
image_size = 224
root = '.'
num_channels = 3
fc_input_size = 25088 # nao muito interessante inicializar o valor na mao
# =============================================================================
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')

x_valid = np.load('x_valid.npy')
y_valid = np.load('y_valid.npy')

x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
# =============================================================================
train_batches_x = np.reshape(x_train, (18, -1, image_size, image_size, 3))
train_batches_y = np.reshape(y_train, (18, -1, 2))

valid_batches_x = np.reshape(x_valid, (5, -1, image_size, image_size, 3))
valid_batches_y = np.reshape(y_valid, (5, -1, 2))

x_test = x_test[:x_test.shape[0] - 1]
y_test = y_test[:y_test.shape[0] - 1]
test_batches_x = np.reshape(x_test, (4, -1, image_size, image_size, 3))
test_batches_y = np.reshape(y_test, (4, -1, 2))

print('train_batches_x:', train_batches_x.shape)
print('train_batches_y: ', train_batches_y.shape)
# =============================================================================
weight_file = 'vgg16_weights.npz'
class_names = ['forgery', 'genuine']

weights = np.load(weight_file)
keys = sorted(weights.keys())
for i, k in enumerate(keys):
    print (i, k, np.shape(weights[k]))
# =============================================================================
graph = tf.Graph()

with graph.as_default():
    
    # keep probability for dropout
    keep_prob = tf.placeholder(tf.float32)
    
    # inputs
    data = tf.placeholder(tf.float32, shape=(None, image_size, image_size,
                                             num_channels))
    labels = tf.placeholder(tf.float32, shape=(None, num_classes))
    
    # conv1_1 parameters
    conv1_1_weights = tf.Variable(weights['conv1_1_W'], trainable=False)
    conv1_1_biases = tf.Variable(weights['conv1_1_b'], trainable=False)
    
    # conv1_2 parameters
    conv1_2_weights = tf.Variable(weights['conv1_2_W'], trainable=False)
    conv1_2_biases = tf.Variable(weights['conv1_2_b'], trainable=False)
    
    # conv2_1 parameters
    conv2_1_weights = tf.Variable(weights['conv2_1_W'], trainable=False)
    conv2_1_biases = tf.Variable(weights['conv2_1_b'], trainable=False)
    
    # conv2_2 parameters
    conv2_2_weights = tf.Variable(weights['conv2_2_W'], trainable=False)
    conv2_2_biases = tf.Variable(weights['conv2_2_b'], trainable=False)
    
    # conv3_1 parameters
    conv3_1_weights = tf.Variable(weights['conv3_1_W'], trainable=False)
    conv3_1_biases = tf.Variable(weights['conv3_1_b'], trainable=False)
    
    # conv3_2 parameters
    conv3_2_weights = tf.Variable(weights['conv3_2_W'], trainable=False)
    conv3_2_biases = tf.Variable(weights['conv3_2_b'], trainable=False)
    
    # conv3_3 parameters
    conv3_3_weights = tf.Variable(weights['conv3_3_W'], trainable=False)
    conv3_3_biases = tf.Variable(weights['conv3_3_b'], trainable=False)
    
    # conv4_1 parameters
    conv4_1_weights = tf.Variable(weights['conv4_1_W'], trainable=False)
    conv4_1_biases = tf.Variable(weights['conv4_1_b'], trainable=False)
    
    # conv4_2 parameters
    conv4_2_weights = tf.Variable(weights['conv4_2_W'], trainable=False)
    conv4_2_biases = tf.Variable(weights['conv4_2_b'], trainable=False)
    
    # conv4_3 parameters
    conv4_3_weights = tf.Variable(weights['conv4_3_W'], trainable=False)
    conv4_3_biases = tf.Variable(weights['conv4_3_b'], trainable=False)
    
    # conv5_1 parameters
    conv5_1_weights = tf.Variable(weights['conv5_1_W'], trainable=False)
    conv5_1_biases = tf.Variable(weights['conv5_1_b'], trainable=False)
    
    # conv5_2 parameters
    conv5_2_weights = tf.Variable(weights['conv5_2_W'], trainable=False)
    conv5_2_biases = tf.Variable(weights['conv5_2_b'], trainable=False)
    
    # conv5_3 parameters
    conv5_3_weights = tf.Variable(weights['conv5_3_W'], trainable=False)
    conv5_3_biases = tf.Variable(weights['conv5_3_b'], trainable=False)
    
    # fc1 parameters
    fc1_weights = tf.Variable(weights['fc6_W'], trainable=True)
    fc1_biases = tf.Variable(weights['fc6_b'], trainable=True)
    
    # fc2 parameters
    fc2_weights = tf.Variable(weights['fc7_W'], trainable=True)
    fc2_biases = tf.Variable(weights['fc7_b'], trainable=True)
    
    # fc3 parameters
    fc3_weights = tf.Variable(weights['fc8_W'], trainable=True)
    fc3_biases = tf.Variable(weights['fc8_b'], trainable=True)
    
    # fc4 parameters (added for the signature problem)
    fc4_weights = tf.Variable(tf.truncated_normal([1000, 2], stddev=0.1),
                              trainable=True)
    fc4_biases = tf.Variable(tf.zeros([2]), trainable=True)
    
    # build the model
    def model(data):
        # convolutional layers
        conv1_1 = tf.nn.conv2d(data, conv1_1_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv1_1 = tf.nn.bias_add(conv1_1, conv1_1_biases)
        conv1_1 = tf.nn.relu(conv1_1)
        
        conv1_2 = tf.nn.conv2d(conv1_1, conv1_2_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv1_2 = tf.nn.bias_add(conv1_2, conv1_2_biases)
        conv1_2 = tf.nn.relu(conv1_2)
        
        pool1 = tf.nn.max_pool(conv1_2, ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1], padding='SAME') # tentar remover o padding
        
        conv2_1 = tf.nn.conv2d(pool1, conv2_1_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv2_1 = tf.nn.bias_add(conv2_1, conv2_1_biases)
        conv2_1 = tf.nn.relu(conv2_1)
        
        conv2_2 = tf.nn.conv2d(conv2_1, conv2_2_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv2_2 = tf.nn.bias_add(conv2_2, conv2_2_biases)
        conv2_2 = tf.nn.relu(conv2_2)
        
        pool2 = tf.nn.max_pool(conv2_2, ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1], padding='SAME') # tentar remover o padding
        
        conv3_1 = tf.nn.conv2d(pool2, conv3_1_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv3_1 = tf.nn.bias_add(conv3_1, conv3_1_biases)
        conv3_1 = tf.nn.relu(conv3_1)
        
        conv3_2 = tf.nn.conv2d(conv3_1, conv3_2_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv3_2 = tf.nn.bias_add(conv3_2, conv3_2_biases)
        conv3_2 = tf.nn.relu(conv3_2)
        
        conv3_3 = tf.nn.conv2d(conv3_2, conv3_3_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv3_3 = tf.nn.bias_add(conv3_3, conv3_3_biases)
        conv3_3 = tf.nn.relu(conv3_3)
        
        pool3 = tf.nn.max_pool(conv3_3, ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1], padding='SAME') # tentar remover o padding
        
        conv4_1 = tf.nn.conv2d(pool3, conv4_1_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv4_1 = tf.nn.bias_add(conv4_1, conv4_1_biases)
        conv4_1 = tf.nn.relu(conv4_1)
        
        conv4_2 = tf.nn.conv2d(conv4_1, conv4_2_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv4_2 = tf.nn.bias_add(conv4_2, conv4_2_biases)
        conv4_2 = tf.nn.relu(conv4_2)
        
        conv4_3 = tf.nn.conv2d(conv4_2, conv4_3_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv4_3 = tf.nn.bias_add(conv4_3, conv4_3_biases)
        conv4_3 = tf.nn.relu(conv4_3)
        
        pool4 = tf.nn.max_pool(conv4_3, ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1], padding='SAME') # tentar remover o padding
        
        conv5_1 = tf.nn.conv2d(pool4, conv5_1_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv5_1 = tf.nn.bias_add(conv5_1, conv5_1_biases)
        conv5_1 = tf.nn.relu(conv5_1)
        
        conv5_2 = tf.nn.conv2d(conv5_1, conv5_2_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv5_2 = tf.nn.bias_add(conv5_2, conv5_2_biases)
        conv5_2 = tf.nn.relu(conv5_2)
        
        conv5_3 = tf.nn.conv2d(conv5_2, conv5_3_weights, [1, 1, 1, 1],
                               padding='SAME')
        conv5_3 = tf.nn.bias_add(conv5_3, conv5_3_biases)
        conv5_3 = tf.nn.relu(conv5_3)
        
        pool5 = tf.nn.max_pool(conv5_3, ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1], padding='SAME') # tentar remover o padding
               
        # fully connected layers
        # fc1
        pool5_flat = tf.reshape(pool5, [-1, fc_input_size])
        fc1 = tf.matmul(pool5_flat, fc1_weights)
        fc1 = tf.nn.bias_add(fc1, fc1_biases)
        fc1= tf.nn.dropout(fc1, keep_prob=keep_prob)
        fc1 = tf.nn.relu(fc1)
        
        # fc2
        fc2 = tf.matmul(fc1, fc2_weights)
        fc2 = tf.nn.bias_add(fc2, fc2_biases)
        fc2 = tf.nn.dropout(fc2, keep_prob=keep_prob)
        fc2 = tf.nn.relu(fc2)
        
        # fc3
        fc3 = tf.matmul(fc2, fc3_weights)
        fc3 = tf.nn.bias_add(fc3, fc3_biases)
        fc3 = tf.nn.dropout(fc3, keep_prob=keep_prob)
        fc3 = tf.nn.relu(fc3)
        
        # fc4
        fc4 = tf.matmul(fc3, fc4_weights)
        fc4 = tf.nn.bias_add(fc4, fc4_biases)
        return tf.nn.dropout(fc4, keep_prob=keep_prob)
    
    
    # optimizing
    beta = 0.002
    logits = model(data)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    regularizer = tf.nn.l2_loss(conv1_1_weights) + tf.nn.l2_loss(conv1_2_weights) + tf.nn.l2_loss(conv2_1_weights) + tf.nn.l2_loss(conv2_2_weights) + tf.nn.l2_loss(conv3_1_weights) + tf.nn.l2_loss(conv3_2_weights) + tf.nn.l2_loss(conv3_3_weights) + tf.nn.l2_loss(conv4_1_weights) + tf.nn.l2_loss(conv4_2_weights) + tf.nn.l2_loss(conv4_3_weights) + tf.nn.l2_loss(conv5_1_weights) + tf.nn.l2_loss(conv5_2_weights) + tf.nn.l2_loss(conv5_3_weights) + tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc3_weights) + tf.nn.l2_loss(fc4_weights)# l2 regularization
    
    loss = tf.reduce_mean(loss + beta * regularizer)
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    # prediction and accuracy
    prediction = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    model_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# =============================================================================
num_steps = 40
validation_accuracies = list()
train_losses = list()
valid_losses = list()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(graph=graph, config=config) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        for batch in range(train_batches_x.shape[0]):
            batch_data = train_batches_x[batch, :, :, :, :]
            batch_label = train_batches_y[batch, :, :]
            _, l, acc = session.run([optimizer, loss, model_accuracy], 
                                    feed_dict={data:batch_data, 
                                               labels:batch_label, 
                                               keep_prob:0.5})
            if (batch % 3 == 0):
                print('loss at epoch %d batch %d: %f' % (step, batch, l))
                
                acc = 0
                for i in range(valid_batches_x.shape[0]):
                    acc += session.run([model_accuracy], feed_dict={data:valid_batches_x[i],
                                      labels:valid_batches_y[i], keep_prob:1})[0]
                acc /= valid_batches_x.shape[0]
                train_losses.append(l)
                validation_accuracies.append(acc)
                print('valid accuracy:', validation_accuracies[-1])
                print()
    # print('Test accuracy:', session.run([model_accuracy], feed_dict={data:x_test, labels:y_test}))
    
    test_predictions = list()
    for test in test_batches_x:
         temp = session.run([prediction], feed_dict={data:test, keep_prob:1})
         test_predictions += temp

np.save('train_losses.npy', train_losses)
np.save('validation_accuracies.npy', validation_accuracies)
np.save('test_predictions.npy', test_predictions)